# Modeling with Keras

In this notebook, I will try to use Tensorflow to build a Neural Network that performs better than the GLM results.
                                                                                    

In [8]:
# Import libraries

import pandas as pd
import numpy as np

from keras.layers import Dense
from keras.models import Sequential
from tensorflow.data.experimental import make_csv_dataset

In [15]:
dfInput = pd.read_csv('../output/dataReady.csv', index_col = 'idx', nrows = 1)

columns = [column for column in dfInput.columns if column not in ['Set', 'idx']]

dfTrain = dfInput.loc[dfInput.Set == 'train']
dfVal = dfInput.loc[dfInput.Set == 'valid']

I want to practice with objects of the type https://www.tensorflow.org/guide/data, to be as analogous to `PyTorch` `Dataloader` as possible.  
https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_csv_dataset  
https://www.tensorflow.org/tutorials/load_data/csv#using_tfdata

In [16]:
tensorTrain = make_csv_dataset('../output/dataReady.csv',
                              label_name = 'quality',
                              batch_size = 512,
                              select_columns = columns)

(OrderedDict([('fixed acidity', <tf.Tensor: shape=(512,), dtype=float32, numpy=
array([10.7,  7.8,  7.8,  8.3,  8.2,  7.5,  6.2,  7. ,  7.2,  7.1,  8. ,
       11.3,  7.8, 11.3,  9.6,  7.5,  7. ,  9.2,  6.5,  7.2,  8. ,  7. ,
        7.1,  6.5,  7.1,  9.5, 10.3,  7.5,  7.9,  8.2,  9. ,  6.9,  7.7,
        7.5,  8.3,  7.7,  7.2,  8.8,  8.5, 10.4,  8.4,  9.8, 10.4,  8.1,
       10.7,  6.5,  6.6,  6.5,  7.6,  8.7, 11.9,  8. ,  6.6,  7. ,  9. ,
        7.1,  7.3, 10. ,  9.5, 10.3,  9.4,  6.6, 15.5,  8. ,  6.5,  7.4,
        7.4,  5.6,  5.9,  6.9,  7.4,  8.9, 10.8,  9.7, 11.9,  8.3,  9.8,
       10.6,  8.3,  7.6,  9. ,  6.5, 10.8,  9. ,  5. ,  9. ,  8.8,  8.7,
        9.4,  5.4, 10.9,  6.2,  9.5,  5.6,  9.9, 10.9,  7.8,  7.5,  7.9,
       11.1,  6.9,  6.9,  6.8,  7.3,  6.3,  9.8,  7.9,  9. ,  7.5,  6.8,
       10.3, 10.4,  8.3,  5.5,  8.1,  8.9,  7.5, 12.4,  7.3, 10.5,  8.9,
        6.4,  8.7,  9.4,  5.4,  8.7,  6.3,  6.3,  6. ,  8.3,  5.2,  6.6,
        8.7,  8. ,  7.3,  8.2,  5.9,  7.3,  

AttributeError: 'tuple' object has no attribute 'items'